### Adding Memory to Chatbots ###

Following topics will be covered as part of today's video. 
1. We will build a small chatbot with Langchain and Openai. 
2. We will then add memory to the chatbot using various langchain modules. 
3. ChatMessageHistory
4. ConversationBufferMemory
5. ConversationSummaryMemory

In [1]:
import os
import langchain

from dotenv import load_dotenv, find_dotenv



In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate


In [3]:
llm = ChatOpenAI(temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

# create a chat prompt template

prompt_template = ChatPromptTemplate.from_messages([
    ("system", " you are a helpful assistant who knows a lot about Apple as a company."),
    ("human", "Respond to this question : {question}")

])

full_prompt = prompt_template.format_messages(question="What is Apple's mission statement?")
llm.invoke(full_prompt)

AIMessage(content='Apple\'s mission statement is: "Apple designs Macs, the best personal computers in the world, along with OS X, iLife, iWork and professional software. Apple leads the digital music revolution with its iPods and iTunes online store. Apple has reinvented the mobile phone with its revolutionary iPhone and App Store, and is defining the future of mobile media and computing devices with iPad."', response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 38, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-aab4a9c2-3692-41c9-a2e1-0326390c708b-0')

In [4]:
print(full_prompt)

[SystemMessage(content=' you are a helpful assistant who knows a lot about Apple as a company.'), HumanMessage(content="Respond to this question : What is Apple's mission statement?")]


In [5]:
full_prompt = prompt_template.format_messages(question="What did i ask earlier?")
llm.invoke(full_prompt)


AIMessage(content='Earlier, you asked about Apple as a company and its history, products, or any other related information. How can I assist you further with your query about Apple?', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 37, 'total_tokens': 70}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-e258acf6-1f36-43aa-b0e2-73e4a11b3de5-0')

In [6]:
### ChatMessageHistory

from langchain.memory import ChatMessageHistory

from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0)
history = ChatMessageHistory()

history.add_ai_message("Hi Ask me anything about Apple as a company")
history.add_user_message("What is Apple's mission statement?")

history.messages

[AIMessage(content='Hi Ask me anything about Apple as a company'),
 HumanMessage(content="What is Apple's mission statement?")]

In [7]:
response = chat.invoke(history.messages)
history.add_ai_message(response)

In [8]:
history.messages

[AIMessage(content='Hi Ask me anything about Apple as a company'),
 HumanMessage(content="What is Apple's mission statement?"),
 AIMessage(content='Apple\'s mission statement is: "Apple designs Macs, the best personal computers in the world, along with OS X, iLife, iWork and professional software. Apple leads the digital music revolution with its iPods and iTunes online store. Apple has reinvented the mobile phone with its revolutionary iPhone and App Store, and is defining the future of mobile media and computing devices with iPad."', response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 27, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_caf95bb1ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-df2bf37c-05bc-42c0-a753-82ad3c6ff78e-0')]

In [9]:
history.add_user_message("Can you summarize your last response")
history.add_ai_message(chat.invoke(history.messages))

history.messages

[AIMessage(content='Hi Ask me anything about Apple as a company'),
 HumanMessage(content="What is Apple's mission statement?"),
 AIMessage(content='Apple\'s mission statement is: "Apple designs Macs, the best personal computers in the world, along with OS X, iLife, iWork and professional software. Apple leads the digital music revolution with its iPods and iTunes online store. Apple has reinvented the mobile phone with its revolutionary iPhone and App Store, and is defining the future of mobile media and computing devices with iPad."', response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 27, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_caf95bb1ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-df2bf37c-05bc-42c0-a753-82ad3c6ff78e-0'),
 HumanMessage(content='Can you summarize your last response'),
 AIMessage(content="Apple's mission is to design the best personal computers, software, and digital devices like iPods, iPhones,

In [10]:
history.add_user_message("What was the first thing i asked you?")

chat.invoke(history.messages)

AIMessage(content="You asked about Apple's mission statement.", response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 170, 'total_tokens': 178}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8b69c01-2c5e-4956-ace5-2552b72383d6-0')

In [11]:
## ConversationBufferMemory ##

from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI, OpenAI
from langchain.chains import ConversationChain


In [12]:
chat = OpenAI(temperature=0)
memory = ConversationBufferMemory(size=5)  # sliding window buffer of size 5
buffer_chain = ConversationChain(llm=chat, memory=memory, verbose=True)

In [13]:
buffer_chain.predict(input = "What is Apple's mission statement?")
buffer_chain.predict(input="Summarize your last response")
buffer_chain.predict(input="Summarize it in even lesser words")
buffer_chain.predict(input="What was the first thing i asked you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is Apple's mission statement?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is Apple's mission statement?
AI:  Apple's mission statement is "to bring the best user experience to its customers through its innovative hardware, software, and services." This statement reflects the company's focus on creating products that are user-friendly and seamlessly

" You asked me about Apple's mission statement."

In [14]:
### ConversationSummaryMemory ###

from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI, OpenAI

In [15]:
chat = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")
memory = ConversationSummaryMemory(llm=chat)

summary_chain = ConversationChain(llm=chat, memory=memory, verbose=True)

In [16]:
summary_chain.predict(input = "What is Apple's mission statement?")
summary_chain.predict(input="Summarize your last response")
summary_chain.predict(input="Summarize it in even lesser words")
summary_chain.predict(input="What was the first thing i asked you?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is Apple's mission statement?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks about Apple's mission statement. The AI explains that it is "to bring the best user experience to its customers through its innovative hardware, software, and services." This reflects the company's focus on user-friendly products, pushing technology boundaries, and provid

" You asked about Apple's mission statement. It is focused on providing the best user experience through innovative hardware, software, and services. This reflects the company's dedication to user-friendly products, pushing technology boundaries, and providing excellent customer service. In summary, Apple's mission is to create user-friendly products, push technology boundaries, and provide excellent customer service."

In [17]:
summary_chain.predict(input = "What is Apple's mission statement?")
summary_chain.predict(input="In your view Is it better than Google's?")
summary_chain.predict(input="Why do you think so?")
summary_chain.predict(input="What can be done to change the opinion?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks about Apple's mission statement and the AI explains that it is focused on providing the best user experience through innovative hardware, software, and services. This reflects the company's dedication to user-friendly products, pushing technology boundaries, and providing excellent customer service. The AI then summarizes its previous response, emphasizing Apple's mission statement and its key components. The AI further simplifies the summary, stating that Apple's mission is to create user-friendly products, push technology boundaries, and provide excellent customer service. The human then asks the AI to repeat the first question, to which the AI respon

" I'm not sure what opinion you are referring to. Can you please clarify?"